In [1]:
import pandas as pd
import numpy as np
import ast

In [2]:
# CSVファイルを読み込む
df_image_embeddings = pd.read_csv('image_embeddings.csv')

# 文字列として保存された埋め込みベクトルをnumpy配列に変換
df_image_embeddings['embedding'] = df_image_embeddings['embedding'].apply(lambda x: np.array(ast.literal_eval(x)))

# 確認
print(f"データ数: {len(df_image_embeddings)}")
print(f"埋め込みベクトルの次元: {len(df_image_embeddings['embedding'].iloc[0])}")

データ数: 1445
埋め込みベクトルの次元: 1024


In [3]:
# データの確認
display(df_image_embeddings.head().style
    .set_properties(**{'text-align': 'left'})
    .set_table_styles([
        {'selector': 'th', 'props': [('text-align', 'left'), ('font-weight', 'bold'), ('white-space', 'nowrap')]},
        {'selector': '.row_heading, .blank', 'props': [('display', 'none')]},
        {'selector': 'td', 'props': [('padding', '5px')]}
    ]))

,image_path,label,embedding
0,datasets\floodnet\FloodNet-Supervised_v1.0\train\train-org-img\6279.jpg,non-flooded,[ 0.02572632 -0.03109741 0.01831055 ... -0.02651977 -0.03256226 -0.01876831]
1,datasets\floodnet\FloodNet-Supervised_v1.0\train\train-org-img\6287.jpg,non-flooded,[ 0.00323868 -0.03903198 0.00982666 ... -0.00110531 -0.03105164 -0.01791382]
2,datasets\floodnet\FloodNet-Supervised_v1.0\train\train-org-img\6332.jpg,non-flooded,[ 0.016922 -0.02352905 0.02267456 ... -0.00616074 -0.01163483 -0.02653503]
3,datasets\floodnet\FloodNet-Supervised_v1.0\train\train-org-img\6334.jpg,non-flooded,[ 0.00160885 -0.02201843 0.01094055 ... 0.00256348 -0.01311493 -0.02182007]
4,datasets\floodnet\FloodNet-Supervised_v1.0\train\train-org-img\6335.jpg,non-flooded,[ 0.00808716 -0.02084351 0.0138855 ... -0.01771545 -0.01469421 -0.0254364 ]


In [4]:
# ラベルの分布確認
print(df_image_embeddings['label'].value_counts())

label
non-flooded    1263
flooded         182
Name: count, dtype: int64


In [5]:
class_captions = [
    'A satellite image of a non-flooded area of land.',
    'A satellite image of a flooded area of land.'
    ]


In [6]:
class_labels = [
    'non-flooded',
    'flooded'
    ]

In [7]:
import cohere
import os
from dotenv import load_dotenv, find_dotenv

In [8]:
_= load_dotenv(find_dotenv())

In [9]:
api_key = os.getenv("COHERE_API_KEY")
model_id = os.getenv("COHERE_EMBED_MODEL_ID")
model_id

'embed-english-v3.0'

In [10]:
co = cohere.Client(api_key=api_key)

In [11]:
ret = co.embed(
    input_type="classification",
    texts=class_captions,
    model=model_id,
    embedding_types=["float"],
)
class_embeddings = np.array(ret.embeddings.float)

In [12]:
class_embeddings.shape

(2, 1024)

In [13]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.special import softmax

In [14]:
# 画像の埋め込みベクトルをスタック
image_embeddings = np.stack(df_image_embeddings['embedding'].values)

# コサイン類似度を計算
similarities = cosine_similarity(image_embeddings, class_embeddings)

# softmaxで確率に変換
probabilities = softmax(similarities, axis=1)

# 予測クラスを取得（0: non-flooded, 1: flooded）
predicted_classes = np.argmax(probabilities, axis=1)

# 結果をデータフレームに追加
df_image_embeddings['predicted_label'] = [class_labels[i] for i in predicted_classes]
df_image_embeddings['confidence'] = np.max(probabilities, axis=1)

In [15]:
# 結果の確認
display(df_image_embeddings.head().style
    .set_properties(**{'text-align': 'left'})
    .set_table_styles([
        {'selector': 'th', 'props': [('text-align', 'left'), ('font-weight', 'bold'), ('white-space', 'nowrap')]},
        {'selector': '.row_heading, .blank', 'props': [('display', 'none')]},
        {'selector': 'td', 'props': [('padding', '5px')]}
    ]))

,image_path,label,embedding,predicted_label,confidence
0,datasets\floodnet\FloodNet-Supervised_v1.0\train\train-org-img\6279.jpg,non-flooded,[ 0.02572632 -0.03109741 0.01831055 ... -0.02651977 -0.03256226 -0.01876831],flooded,0.511935
1,datasets\floodnet\FloodNet-Supervised_v1.0\train\train-org-img\6287.jpg,non-flooded,[ 0.00323868 -0.03903198 0.00982666 ... -0.00110531 -0.03105164 -0.01791382],flooded,0.500765
2,datasets\floodnet\FloodNet-Supervised_v1.0\train\train-org-img\6332.jpg,non-flooded,[ 0.016922 -0.02352905 0.02267456 ... -0.00616074 -0.01163483 -0.02653503],non-flooded,0.516070
3,datasets\floodnet\FloodNet-Supervised_v1.0\train\train-org-img\6334.jpg,non-flooded,[ 0.00160885 -0.02201843 0.01094055 ... 0.00256348 -0.01311493 -0.02182007],non-flooded,0.515104
4,datasets\floodnet\FloodNet-Supervised_v1.0\train\train-org-img\6335.jpg,non-flooded,[ 0.00808716 -0.02084351 0.0138855 ... -0.01771545 -0.01469421 -0.0254364 ],non-flooded,0.515276


In [16]:
# 正解率の確認
accuracy = (df_image_embeddings['label'] == df_image_embeddings['predicted_label']).mean()
print(f"\n正解率: {accuracy:.3f}")


正解率: 0.842


In [17]:
# 混同行列の要素を計算
true_positives = ((df_image_embeddings['label'] == 'flooded') & 
                 (df_image_embeddings['predicted_label'] == 'flooded')).sum()
true_negatives = ((df_image_embeddings['label'] == 'non-flooded') & 
                  (df_image_embeddings['predicted_label'] == 'non-flooded')).sum()
false_positives = ((df_image_embeddings['label'] == 'non-flooded') & 
                  (df_image_embeddings['predicted_label'] == 'flooded')).sum()
false_negatives = ((df_image_embeddings['label'] == 'flooded') & 
                  (df_image_embeddings['predicted_label'] == 'non-flooded')).sum()

# 適合率（Precision）= TP / (TP + FP)
precision = true_positives / (true_positives + false_positives)

# 再現率（Recall）= TP / (TP + FN)
recall = true_positives / (true_positives + false_negatives)

# F1スコア = 2 * (precision * recall) / (precision + recall)
f1_score = 2 * (precision * recall) / (precision + recall)

print(f"適合率（Precision）: {precision:.3f}")
print(f"再現率（Recall）: {recall:.3f}")
print(f"F1スコア: {f1_score:.3f}")
print("\n詳細:")
print(f"True Positives (正しく浸水と予測): {true_positives}")
print(f"True Negatives (正しく非浸水と予測): {true_negatives}")
print(f"False Positives (誤って浸水と予測): {false_positives}")
print(f"False Negatives (浸水を見逃した数): {false_negatives}")
print(f"実際の浸水画像の総数: {true_positives + false_negatives}")
print(f"実際の非浸水画像の総数: {true_negatives + false_positives}")
print(f"総数: {len(df_image_embeddings)}")

適合率（Precision）: 0.439
再現率（Recall）: 0.929
F1スコア: 0.596

詳細:
True Positives (正しく浸水と予測): 169
True Negatives (正しく非浸水と予測): 1047
False Positives (誤って浸水と予測): 216
False Negatives (浸水を見逃した数): 13
実際の浸水画像の総数: 182
実際の非浸水画像の総数: 1263
総数: 1445


In [18]:
columns_to_save = ['image_path', 'label', 'predicted_label', 'confidence']
df_image_embeddings[columns_to_save].to_csv('classified.csv', index=False)